# XGBoost training
This is an auto-generated notebook. To reproduce these results, attach this notebook to the **Pocclouds** cluster and rerun it.
- Compare trials in the [MLflow experiment](#mlflow/experiments/4207461175781303/s?orderByKey=metrics.%60val_roc_auc_score%60&orderByAsc=false)
- Navigate to the parent notebook [here](#notebook/4207461175781304) (If you launched the AutoML experiment using the Experiments UI, this link isn't very useful.)
- Clone this notebook into your project folder by selecting **File > Clone** in the notebook toolbar.

Runtime Version: _10.4.x-cpu-ml-scala2.12_

In [0]:
import mlflow
import databricks.automl_runtime

target_col = "Churn"

## Load Data

In [0]:
from mlflow.tracking import MlflowClient
import os
import uuid
import shutil
import pandas as pd

# Create temp directory to download input data from MLflow
input_temp_dir = os.path.join(os.environ["SPARK_LOCAL_DIRS"], "tmp", str(uuid.uuid4())[:8])
os.makedirs(input_temp_dir)


# Download the artifact and read it into a pandas DataFrame
input_client = MlflowClient()
input_data_path = input_client.download_artifacts("98691adf863b40b6a0d9d4103d06fa2d", "data", input_temp_dir)

df_loaded = pd.read_parquet(os.path.join(input_data_path, "training_data"))
# Delete the temp data
shutil.rmtree(input_temp_dir)

# Preview data
df_loaded.head(5)

,Tenure,WarehouseToHome,HourSpendOnApp,NumberOfDeviceRegistered,SatisfactionScore,NumberOfAddress,Complain,OrderAmountHikeFromlastYear,CouponUsed,OrderCount,...,PreferedOrderCat_Fashion,PreferedOrderCat_Grocery,PreferedOrderCat_Laptop_and_Accessory,PreferedOrderCat_Mobile,PreferedOrderCat_Mobile_Phone,PreferedOrderCat_Others,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Churn
0,5.0,7.0,4.0,4.0,2.0,10.0,1.0,12.0,2.0,2.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1
1,1.0,9.0,4.0,5.0,3.0,8.0,1.0,16.0,1.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1
2,1.0,31.0,3.0,5.0,3.0,7.0,1.0,15.0,1.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1
3,1.0,16.0,3.0,5.0,5.0,9.0,0.0,24.0,1.0,2.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1
4,1.0,13.0,3.0,4.0,5.0,4.0,0.0,12.0,2.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1


### Select supported columns
Select only the columns that are supported. This allows us to train a model that can predict on a dataset that has extra columns that are not used in training.
`[]` are dropped in the pipelines. See the Alerts tab of the AutoML Experiment page for details on why these columns are dropped.

In [0]:
from databricks.automl_runtime.sklearn.column_selector import ColumnSelector
supported_cols = ["PreferredPaymentMode_UPI", "PreferedOrderCat_Others", "PreferedOrderCat_Mobile_Phone", "PreferedOrderCat_Fashion", "Complain", "PreferredPaymentMode_CC", "MaritalStatus_Divorced", "PreferredPaymentMode_E_wallet", "MaritalStatus_Single", "PreferredPaymentMode_Debit_Card", "PreferredPaymentMode_Credit_Card", "PreferedOrderCat_Laptop_and_Accessory", "Gender_Male", "CashbackAmount", "DaySinceLastOrder", "PreferedOrderCat_Mobile", "Gender_Female", "PreferredLoginDevice_Phone", "HourSpendOnApp", "WarehouseToHome", "CityTier_3", "NumberOfAddress", "PreferredLoginDevice_Computer", "PreferredPaymentMode_COD", "MaritalStatus_Married", "SatisfactionScore", "Tenure", "OrderAmountHikeFromlastYear", "OrderCount", "PreferedOrderCat_Grocery", "CouponUsed", "CityTier_1", "PreferredLoginDevice_Mobile_Phone", "NumberOfDeviceRegistered", "PreferredPaymentMode_Cash_on_Delivery", "CityTier_2"]
col_selector = ColumnSelector(supported_cols)

## Preprocessors

In [0]:
transformers = []

### Numerical columns

Missing values for numerical columns are imputed with mean by default.

In [0]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

num_imputers = []
num_imputers.append(("impute_mean", SimpleImputer(), ["CashbackAmount", "DaySinceLastOrder", "OrderAmountHikeFromlastYear", "Tenure", "WarehouseToHome"]))

numerical_pipeline = Pipeline(steps=[
    ("converter", FunctionTransformer(lambda df: df.apply(pd.to_numeric, errors="coerce"))),
    ("imputers", ColumnTransformer(num_imputers, sparse_threshold=0))
])

transformers.append(("numerical", numerical_pipeline, ["WarehouseToHome", "CashbackAmount", "DaySinceLastOrder", "Tenure", "OrderAmountHikeFromlastYear"]))

### Categorical columns

#### Low-cardinality categoricals
Convert each low-cardinality categorical column into multiple binary columns through one-hot encoding.
For each input categorical column (string or numeric), the number of output columns is equal to the number of unique values in the input column.

In [0]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

one_hot_imputers = []

one_hot_pipeline = Pipeline(steps=[
    ("imputers", ColumnTransformer(one_hot_imputers, sparse_threshold=0, remainder="passthrough")),
    ("one_hot_encoder", OneHotEncoder(handle_unknown="ignore")),
])

transformers.append(("onehot", one_hot_pipeline, ["CityTier_1", "CityTier_2", "CityTier_3", "Complain", "CouponUsed", "Gender_Female", "Gender_Male", "HourSpendOnApp", "MaritalStatus_Divorced", "MaritalStatus_Married", "MaritalStatus_Single", "NumberOfAddress", "NumberOfDeviceRegistered", "OrderCount", "PreferedOrderCat_Fashion", "PreferedOrderCat_Grocery", "PreferedOrderCat_Laptop_and_Accessory", "PreferedOrderCat_Mobile", "PreferedOrderCat_Mobile_Phone", "PreferedOrderCat_Others", "PreferredLoginDevice_Computer", "PreferredLoginDevice_Mobile_Phone", "PreferredLoginDevice_Phone", "PreferredPaymentMode_CC", "PreferredPaymentMode_COD", "PreferredPaymentMode_Cash_on_Delivery", "PreferredPaymentMode_Credit_Card", "PreferredPaymentMode_Debit_Card", "PreferredPaymentMode_E_wallet", "PreferredPaymentMode_UPI", "SatisfactionScore"]))

In [0]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(transformers, remainder="passthrough", sparse_threshold=0)

### Feature standardization
Scale all feature columns to be centered around zero with unit variance.

In [0]:
from sklearn.preprocessing import StandardScaler

standardizer = StandardScaler()

## Train - Validation - Test Split
Split the input data into 3 sets:
- Train (60% of the dataset used to train the model)
- Validation (20% of the dataset used to tune the hyperparameters of the model)
- Test (20% of the dataset used to report the true performance of the model on an unseen dataset)

In [0]:
from sklearn.model_selection import train_test_split

split_X = df_loaded.drop([target_col], axis=1)
split_y = df_loaded[target_col]

# Split out train data
X_train, split_X_rem, y_train, split_y_rem = train_test_split(split_X, split_y, train_size=0.6, random_state=9871075, stratify=split_y)

# Split remaining data equally for validation and test
X_val, X_test, y_val, y_test = train_test_split(split_X_rem, split_y_rem, test_size=0.5, random_state=9871075, stratify=split_y_rem)

## Train classification model
- Log relevant metrics to MLflow to track runs
- All the runs are logged under [this MLflow experiment](#mlflow/experiments/4207461175781303/s?orderByKey=metrics.%60val_roc_auc_score%60&orderByAsc=false)
- Change the model parameters and re-run the training cell to log a different trial to the MLflow experiment
- To view the full list of tunable hyperparameters, check the output of the cell below

In [0]:
from xgboost import XGBClassifier

help(XGBClassifier)

Help on class XGBClassifier in module xgboost.sklearn:

class XGBClassifier(XGBModel, sklearn.base.ClassifierMixin)
 |  XGBClassifier(*, objective: Union[str, Callable[[numpy.ndarray, numpy.ndarray], Tuple[numpy.ndarray, numpy.ndarray]], NoneType] = 'binary:logistic', use_label_encoder: bool = True, **kwargs: Any) -> None
 |  
 |  Implementation of the scikit-learn API for XGBoost classification.
 |  
 |  
 |  Parameters
 |  ----------
 |  
 |      n_estimators : int
 |          Number of boosting rounds.
 |      use_label_encoder : bool
 |          (Deprecated) Use the label encoder from scikit-learn to encode the labels. For new
 |          code, we recommend that you set this parameter to False.
 |  
 |      max_depth :  Optional[int]
 |          Maximum tree depth for base learners.
 |      learning_rate : Optional[float]
 |          Boosting learning rate (xgb's "eta")
 |      verbosity : Optional[int]
 |          The degree of verbosity. Valid values are 0 (silent) - 3 (debug).
 

In [0]:
import mlflow
import sklearn
from sklearn import set_config
from sklearn.pipeline import Pipeline

set_config(display="diagram")

xgbc_classifier = XGBClassifier(
  colsample_bytree=0.44193345569952586,
  learning_rate=0.06282864067255331,
  max_depth=5,
  min_child_weight=18,
  n_estimators=771,
  n_jobs=100,
  subsample=0.5933268876765794,
  verbosity=0,
  random_state=9871075,
)

model = Pipeline([
    ("column_selector", col_selector),
    ("preprocessor", preprocessor),
    ("standardizer", standardizer),
    ("classifier", xgbc_classifier),
])

# Create a separate pipeline to transform the validation dataset. This is used for early stopping.
pipeline = Pipeline([
    ("column_selector", col_selector),
    ("preprocessor", preprocessor),
    ("standardizer", standardizer),
])

mlflow.sklearn.autolog(disable=True)
pipeline.fit(X_train, y_train)
X_val_processed = pipeline.transform(X_val)

model

Pipeline Pipeline(steps=[('column_selector',
 ColumnSelector(cols=['PreferredPaymentMode_UPI',
 'PreferedOrderCat_Others',
 'PreferedOrderCat_Mobile_Phone',
 'PreferedOrderCat_Fashion', 'Complain',
 'PreferredPaymentMode_CC',
 'MaritalStatus_Divorced',
 'PreferredPaymentMode_E_wallet',
 'MaritalStatus_Single',
 'PreferredPaymentMode_Debit_Card',
 'PreferredPaymentMode_Credit_Ca...
 learning_rate=0.06282864067255331,
 max_delta_step=None, max_depth=5,
 min_child_weight=18, missing=nan,
 monotone_constraints=None, n_estimators=771,
 n_jobs=100, num_parallel_tree=None,
 predictor=None, random_state=9871075,
 reg_alpha=None, reg_lambda=None,
 scale_pos_weight=None,
 subsample=0.5933268876765794, tree_method=None,
 validate_parameters=None, verbosity=0))]) ColumnSelector ColumnSelector(cols=['PreferredPaymentMode_UPI', 'PreferedOrderCat_Others',
 'PreferedOrderCat_Mobile_Phone',
 'PreferedOrderCat_Fashion', 'Complain',
 'PreferredPaymentMode_CC', 'MaritalStatus_Divorced',
 'PreferredPaymentMode_E_wallet', 'MaritalStatus_Single',
 'PreferredPaymentMode_Debit_Card',
 'PreferredPaymentMode_Credit_Card',
 'PreferedOrderCat_Laptop_and_Ac...
 'CashbackAmount', 'DaySinceLastOrder',
 'PreferedOrderCat_Mobile', 'Gender_Female',
 'PreferredLoginDevice_Phone', 'HourSpendOnApp',
 'WarehouseToHome', 'CityTier_3', 'NumberOfAddress',
 'PreferredLoginDevice_Computer',
 'PreferredPaymentMode_COD', 'MaritalStatus_Married',
 'SatisfactionScore', 'Tenure',
 'OrderAmountHikeFromlastYear', 'OrderCount',
 'PreferedOrderCat_Grocery', ...]) preprocessor: ColumnTransformer ColumnTransformer(remainder='passthrough', sparse_threshold=0,
 transformers=[('numerical',
 Pipeline(steps=[('converter',
 FunctionTransformer(func= at 0x7f11fc9bd670>)),
 ('imputers',
 ColumnTransformer(sparse_threshold=0,
 transformers=[('impute_mean',
 SimpleImputer(),
 ['CashbackAmount',
 'DaySinceLastOrder',
 'OrderAmountHikeFromlastYear',
 'Tenure',...
 'PreferedOrderCat_Others',
 'PreferredLoginDevice_Computer',
 'PreferredLoginDevice_Mobile_Phone',
 'PreferredLoginDevice_Phone',
 'PreferredPaymentMode_CC',
 'PreferredPaymentMode_COD',
 'PreferredPaymentMode_Cash_on_Delivery',
 'PreferredPaymentMode_Credit_Card',
 'PreferredPaymentMode_Debit_Card',
 'PreferredPaymentMode_E_wallet',
 'PreferredPaymentMode_UPI', ...])]) numerical ['WarehouseToHome', 'CashbackAmount', 'DaySinceLastOrder', 'Tenure', 'OrderAmountHikeFromlastYear'] FunctionTransformer FunctionTransformer(func= at 0x7f11fc9bd670>) imputers: ColumnTransformer ColumnTransformer(sparse_threshold=0,
 transformers=[('impute_mean', SimpleImputer(),
 ['CashbackAmount', 'DaySinceLastOrder',
 'OrderAmountHikeFromlastYear', 'Tenure',
 'WarehouseToHome'])]) impute_mean ['CashbackAmount', 'DaySinceLastOrder', 'OrderAmountHikeFromlastYear', 'Tenure', 'WarehouseToHome'] SimpleImputer SimpleImputer() onehot ['CityTier_1', 'CityTier_2', 'CityTier_3', 'Complain', 'CouponUsed', 'Gender_Female', 'Gender_Male', 'HourSpendOnApp', 'MaritalStatus_Divorced', 'MaritalStatus_Married', 'MaritalStatus_Single', 'NumberOfAddress', 'NumberOfDeviceRegistered', 'OrderCount', 'PreferedOrderCat_Fashion', 'PreferedOrderCat_Grocery', 'PreferedOrderCat_Laptop_and_Accessory', 'PreferedOrderCat_Mobile', 'PreferedOrderCat_Mobile_Phone', 'PreferedOrderCat_Others', 'PreferredLoginDevice_Computer', 'PreferredLoginDevice_Mobile_Phone', 'PreferredLoginDevice_Phone', 'PreferredPaymentMode_CC', 'PreferredPaymentMode_COD', 'PreferredPaymentMode_Cash_on_Delivery', 'PreferredPaymentMode_Credit_Card', 'PreferredPaymentMode_Debit_Card', 'PreferredPaymentMode_E_wallet', 'PreferredPaymentMode_UPI', 'SatisfactionScore'] imputers: ColumnTransformer ColumnTransformer(remainder='passthrough', sparse_threshold=0, transformers=[]) remainder passthrough passthrough OneHotEncoder OneHotEncoder(handle_unknown='ignore') remainder [['PreferredPaymentMode_UPI', 'PreferedOrderCat_Others', 'PreferedOrderCat_Mobile_Phone', 'PreferedOrderCat_Fashion', 'Complain', 'PreferredPaymentMode_CC', 

In [0]:
# Enable automatic logging of input samples, metrics, parameters, and models
mlflow.sklearn.autolog(log_input_examples=True, silent=True)

with mlflow.start_run(experiment_id="4207461175781303", run_name="xgboost") as mlflow_run:
    model.fit(X_train, y_train, classifier__early_stopping_rounds=5, classifier__eval_set=[(X_val_processed,y_val)], classifier__verbose=False)
    
    # Training metrics are logged by MLflow autologging
    # Log metrics for the validation set
    xgbc_val_metrics = mlflow.sklearn.eval_and_log_metrics(model, X_val, y_val, prefix="val_")

    # Log metrics for the test set
    xgbc_test_metrics = mlflow.sklearn.eval_and_log_metrics(model, X_test, y_test, prefix="test_")

    # Display the logged metrics
    xgbc_val_metrics = {k.replace("val_", ""): v for k, v in xgbc_val_metrics.items()}
    xgbc_test_metrics = {k.replace("test_", ""): v for k, v in xgbc_test_metrics.items()}
    display(pd.DataFrame([xgbc_val_metrics, xgbc_test_metrics], index=["validation", "test"]))

/databricks/python/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


,precision_score,recall_score,f1_score,accuracy_score,log_loss,roc_auc_score,score
validation,0.895974,0.901421,0.897059,0.901421,0.258924,0.903959,0.901421
test,0.888301,0.894316,0.889893,0.894316,0.257946,0.908636,0.894316


In [0]:
# Patch requisite packages to the model environment YAML for model serving
import os
import shutil
import uuid
import yaml

None

import xgboost
from mlflow.tracking import MlflowClient

xgbc_temp_dir = os.path.join(os.environ["SPARK_LOCAL_DIRS"], str(uuid.uuid4())[:8])
os.makedirs(xgbc_temp_dir)
xgbc_client = MlflowClient()
xgbc_model_env_path = xgbc_client.download_artifacts(mlflow_run.info.run_id, "model/conda.yaml", xgbc_temp_dir)
xgbc_model_env_str = open(xgbc_model_env_path)
xgbc_parsed_model_env_str = yaml.load(xgbc_model_env_str, Loader=yaml.FullLoader)

xgbc_parsed_model_env_str["dependencies"][-1]["pip"].append(f"xgboost=={xgboost.__version__}")

with open(xgbc_model_env_path, "w") as f:
  f.write(yaml.dump(xgbc_parsed_model_env_str))
xgbc_client.log_artifact(run_id=mlflow_run.info.run_id, local_path=xgbc_model_env_path, artifact_path="model")
shutil.rmtree(xgbc_temp_dir)

## Feature importance

SHAP is a game-theoretic approach to explain machine learning models, providing a summary plot
of the relationship between features and model output. Features are ranked in descending order of
importance, and impact/color describe the correlation between the feature and the target variable.
- Generating SHAP feature importance is a very memory intensive operation, so to ensure that AutoML can run trials without
  running out of memory, we disable SHAP by default.<br />
  You can set the flag defined below to `shap_enabled = True` and re-run this notebook to see the SHAP plots.
- To reduce the computational overhead of each trial, a single example is sampled from the validation set to explain.<br />
  For more thorough results, increase the sample size of explanations, or provide your own examples to explain.
- SHAP cannot explain models using data with nulls; if your dataset has any, both the background data and
  examples to explain will be imputed using the mode (most frequent values). This affects the computed
  SHAP values, as the imputed samples may not match the actual data distribution.

For more information on how to read Shapley values, see the [SHAP documentation](https://shap.readthedocs.io/en/latest/example_notebooks/overviews/An%20introduction%20to%20explainable%20AI%20with%20Shapley%20values.html).

In [0]:
# Set this flag to True and re-run the notebook to see the SHAP plots
shap_enabled = False

In [0]:
if shap_enabled:
    from shap import KernelExplainer, summary_plot
    # Sample background data for SHAP Explainer. Increase the sample size to reduce variance.
    train_sample = X_train.sample(n=min(100, len(X_train.index)))

    # Sample a single example from the validation set to explain. Increase the sample size and rerun for more thorough results.
    example = X_val.sample(n=1)

    # Use Kernel SHAP to explain feature importance on the example from the validation set.
    predict = lambda x: model.predict(pd.DataFrame(x, columns=X_train.columns))
    explainer = KernelExplainer(predict, train_sample, link="identity")
    shap_values = explainer.shap_values(example, l1_reg=False)
    summary_plot(shap_values, example, class_names=model.classes_)

## Inference
[The MLflow Model Registry](https://docs.databricks.com/applications/mlflow/model-registry.html) is a collaborative hub where teams can share ML models, work together from experimentation to online testing and production, integrate with approval and governance workflows, and monitor ML deployments and their performance. The snippets below show how to add the model trained in this notebook to the model registry and to retrieve it later for inference.

> **NOTE:** The `model_uri` for the model already trained in this notebook can be found in the cell below

### Register to Model Registry
```
model_name = "Example"

model_uri = f"runs:/{ mlflow_run.info.run_id }/model"
registered_model_version = mlflow.register_model(model_uri, model_name)
```

### Load from Model Registry
```
model_name = "Example"
model_version = registered_model_version.version

model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")
model.predict(input_X)
```

### Load model without registering
```
model_uri = f"runs:/{ mlflow_run.info.run_id }/model"

model = mlflow.pyfunc.load_model(model_uri)
model.predict(input_X)
```

In [0]:
# model_uri for the generated model
print(f"runs:/{ mlflow_run.info.run_id }/model")

runs:/31d12e80a5304aaf9c6e2ab26e74f1a5/model
